In [1]:
import os
import astroNN
from astroNN import datasets
from astroNN.apogee import allstar, visit_spectra, combined_spectra, gap_delete, apogee_continuum, bitmask_boolean
from astroNN.models import load_folder, ApogeeBCNNCensored
from astroNN.apogee.chips import wavelength_solution, chips_split

from astropy.io import fits
from astropy.wcs import WCS
from tensorflow.keras import utils
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm
import pickle
import traceback
import h5py as h5

%matplotlib inline
import matplotlib.ticker as ticker

In [2]:
geir_path = "/geir_data/scr/petershi" #should save work here 
sdssv_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/spectra/star" #path for SDSSV spectra
sdssiv_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/redux/dr17/stars/apo25m"

xmatch_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/ipl-1_dr17_lite_230517_byYS.fits"
allstar_path = "/yngve_data/sdss/apogee/dr17/apogee/spectro/aspcap/dr17/synspec/allStar-dr17-synspec.fits"
catlist_path = "/yngve_data/sdssv/ipl-2/spectro/astra/0.3.0/v6_0_9-1.0/aux_xmatch/allStar-TIC_v8-APOGEE-0.2.6-v6_0_9-1.0.fits"

In [3]:
h5_path = os.path.join(geir_path, "serialized", "Retrain Dataset v2.hdf5")
hf = h5.File(h5_path, 'r')

In [4]:
def getSpectrum(star, ind, mask_value):

    spec = np.array(star["FLUX"])
    spec_err = np.array(star["FLUX_ERR"])
    bitmask = np.array(star["BITMASK"])

    target_bit = [0, 1, 2, 3, 4, 5, 6, 7, 12]

    try:
        _normspec, _spec_err = apogee_continuum(spec, spec_err, 
                                                bitmask=bitmask, 
                                                target_bit=target_bit, 
                                                dr=17, mask_value=mask_value)

        return (_normspec, _spec_err)

    except Exception: # for linalgerror
        #print("LinAlgError encountered for " + str(ind))
        return None

In [5]:
def read_h5(h5):

    keys = list(h5.keys())
    length = len(keys)

    normspec_container = []
    normspec_err_container = []
    elem_container = []
    elem_errs_container = []

    symbols = ['teff', 'logg', 'C', 'CI', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K',
               'Ca', 'Ti', 'TiII', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni']

    try:
        for key in tqdm(keys):

            valid_elems = []
            valid_elem_errs = []

            grp = h5[key]

            spec = grp["Spectra"]
            elems = grp["Elements"]
            elem_errs = grp["Element Errors"]

            container = getSpectrum(spec, key, 1.)

            if container is not None:
                normspec = container[0]
                normspec_err = container[1]

                normspec_container.append(normspec[0])
                normspec_err_container.append(normspec_err[0])

                for sym in symbols:
                    if sym == "P":
                        valid_elems.append(elems["C"][()])
                        valid_elem_errs.append(elem_errs["C"][()])
                    else:
                        valid_elems.append(elems[sym][()])
                        valid_elem_errs.append(elem_errs[sym][()])

                elem_container.append(valid_elems)
                elem_errs_container.append(valid_elem_errs)


                # print(normspec)
                # print(normspec_err)
                # print(valid_elems)
                # print(valid_elem_errs)
    except:
        print(traceback.format_exc())

    return normspec_container, normspec_err_container, elem_container, elem_errs_container
            

In [6]:
x, x_err, y, y_err = read_h5(hf)

  0%|          | 0/98224 [00:00<?, ?it/s]/tmp/ipykernel_43374/502915280.py:3: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
/tmp/ipykernel_43374/502915280.py:4: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
/tmp/ipykernel_43374/502915280.py:5: DeprecationWarning: `product` is deprecated as of NumPy 1.25.0, and will be removed in NumPy 2.0. Please use `prod` instead.
  0%|          | 219/98224 [00:06<46:41, 34.98it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


  1%|          | 662/98224 [00:19<49:55, 32.57it/s]/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/astroNN/apogee/chips.py:252: RuntimeWarning: invalid value encountered in divide
/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/astroNN/apogee/chips.py:253: RuntimeWarning: invalid value encountered in divide
  1%|          | 715/98224 [00:21<45:00, 36.11it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


  1%|          | 1043/98224 [00:31<42:38, 37.98it/s]

...

100%|█████████▉| 98100/98224 [45:25<00:03, 36.28it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


100%|█████████▉| 98180/98224 [45:28<00:01, 33.70it/s]


Intel MKL ERROR: Parameter 4 was incorrect on entry to DGELSD.


100%|██████████| 98224/98224 [45:29<00:00, 35.99it/s]


In [7]:
x_arr = np.array(x, dtype="float32")
y_arr = np.array(y, dtype="float32")
xerr_arr = np.array(x_err, dtype="float32")
yerr_arr =  np.array(y_err, dtype="float32")

In [8]:
yerr_arr = np.nan_to_num(yerr_arr, nan = 1000)

In [10]:
print(
np.count_nonzero(np.isnan(x_arr)),
np.count_nonzero(np.isnan(xerr_arr)),
np.count_nonzero(np.isnan(y_arr)),
np.count_nonzero(np.isnan(yerr_arr))
)

0 0 444 0


In [11]:
hf.close()

In [12]:
bcnncensored_net = ApogeeBCNNCensored()

In [13]:
bcnncensored_net.targetname = ['teff', 'logg', 'C', 'CI', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'P', 'S', 'K',
                               'Ca', 'Ti', 'TiII', 'V', 'Cr', 'Mn', 'Fe', 'Co', 'Ni']

In [14]:
bcnncensored_net.fit(x_arr, y_arr, xerr_arr, yerr_arr)

Number of Training Data: 87789, Number of Validation Data: 9754
====Message from Normalizer====
You selected mode: 3
Featurewise Center: {'input': True, 'input_err': True, 'labels_err': True}
Datawise Center: {'input': False, 'input_err': False, 'labels_err': False} 
Featurewise std Center: {'input': False, 'input_err': False, 'labels_err': False}
Datawise std Center: {'input': False, 'input_err': False, 'labels_err': False} 
====Message ends====
====Message from Normalizer====
You selected mode: 3
Featurewise Center: {'input': True, 'input_err': True, 'labels_err': True}
Datawise Center: {'input': False, 'input_err': False, 'labels_err': False} 
Featurewise std Center: {'input': False, 'input_err': False, 'labels_err': False}
Datawise std Center: {'input': False, 'input_err': False, 'labels_err': False} 
====Message ends====
====Message from Normalizer====
You selected mode: 3
Featurewise Center: {'input': True, 'input_err': True, 'labels_err': True}
Datawise Center: {'input': False, 

2023-07-05 17:17:18.054567: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1635] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11385 MB memory:  -> device: 0, name: TITAN Xp, pci bus id: 0000:04:00.0, compute capability: 6.1
/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer RandomNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.


Epoch 1/50


2023-07-05 17:17:31.789609: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype int32
	 [[{{node Placeholder/_0}}]]
/home/petershi/miniconda3/envs/mainenv/lib/python3.10/site-packages/keras/engine/functional.py:638: UserWarning: Input dict contained keys ['input_err'] which did not match any model input. They will be ignored by the model.
2023-07-05 17:18:07.996399: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600
2023-07-05 17:18:20.839303: I tensorflow/compiler/xla/service/service.cc:169] XLA service 0x7f50a8012cf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-07-05 17:18:20.839378: I tensorflow/compiler/xla/service/service.cc:177]   StreamExecutor device (0): TITAN Xp, Compute Capability 6.

1371/1371 - 250s - loss: -3.7242e-01 - output_loss: -3.7246e-01 - output_mean_absolute_error: 0.3551 - output_mean_error: -4.1039e-02 - variance_output_mean_absolute_error: 2.0157 - variance_output_mean_error: 1.6831 - val_loss: -6.8691e-01 - val_output_loss: -6.8696e-01 - val_output_mean_absolute_error: 0.2629 - val_output_mean_error: -1.5148e-02 - val_variance_output_mean_absolute_error: 2.5890 - val_variance_output_mean_error: 2.3639 - lr: 5.0000e-04 - 250s/epoch - 183ms/step
Epoch 2/50
1371/1371 - 75s - loss: -7.8642e-01 - output_loss: -7.8648e-01 - output_mean_absolute_error: 0.2401 - output_mean_error: -1.7008e-02 - variance_output_mean_absolute_error: 2.8048 - variance_output_mean_error: 2.6184 - val_loss: -8.4970e-01 - val_output_loss: -8.4976e-01 - val_output_mean_absolute_error: 0.2303 - val_output_mean_error: -3.0791e-02 - val_variance_output_mean_absolute_error: 2.9848 - val_variance_output_mean_error: 2.7912 - lr: 5.0000e-04 - 75s/epoch - 54ms/step
Epoch 3/50
1371/1371 - 7

In [15]:
save_path = os.path.join(geir_path, "models", "retrain 2")

In [16]:
bcnncensored_net.save(save_path)

model_weights.h5 saved to /geir_data/scr/petershi/models/retrain 2/model_weights.h5


: 